# Hive Assignment 1. DDL: Create Tables

The purpose of this task is to create an external table on the posts data of the *stackoverflow.com* website.

Create your own database and 'use' it. Create external table `posts_sample_external` over the sample dataset with posts in `/data/stackexchange1000` directory. Create managed table `posts_sample` and populate with the data from the external table. The `posts_sample` table should be partitioned by year and by month of post creation. Provide output of query which selects lines number per each partition in the format:

```
year <tab> month <tab> lines count
```

where year in `YYYY` format and month in `YYYY-MM` format. The result is the 3th line of the last query output.

The result on the sample dataset:

```
2008    2008-10 73
```

**Note:** the external grader requires one file for this task. That's why all step are combined in one .hql file.

## Step 1. Complete the assignment

In [1]:
%%writefile task1_create_external_table.hql
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE stackoverflow_;

-- Create external table posts_sample_external with suitable values
DROP TABLE IF EXISTS posts_sample_external;

CREATE EXTERNAL TABLE posts_sample_external (
    id INT,
    year INT,
    month STRING
)
ROW FORMAT
    SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
        "input.regex" = ".*?(?=\\bId=\"(\\d+)\").*(?=.*\\bCreationDate=\"(\\d{4}).*)(?=.*\\bCreationDate=\"(\\d{4}-\\d{2}).*).*$"
    )
LOCATION '/data/stackexchange1000/posts'
TBLPROPERTIES (
    "skip.header.line.count"="1"
);


Writing task1_create_external_table.hql


In [2]:
!hive -f task1_create_external_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 0.815 seconds
OK
Time taken: 0.236 seconds
OK
Time taken: 0.736 seconds


In [3]:
%%writefile task1_check_select.hql
USE stackoverflow_;

SELECT * FROM posts_sample_external LIMIT 10;

Writing task1_check_select.hql


In [4]:
!hive -f task1_check_select.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
Time taken: 13.075 seconds
OK
1394	2008	2008-08
3543	2008	2008-08
4521	2008	2008-08
8689	2008	2008-08
9062	2008	2008-08
14671	2008	2008-08
16307	2008	2008-08
18780	2008	2008-08
18929	2008	2008-08
19668	2008	2008-08
Time taken: 2.907 seconds, Fetched: 10 row(s)


In [5]:
%%writefile task1_create_managed_table.hql
USE stackoverflow_;

-- Create managed table posts_sample partitioned by year and month
CREATE TABLE posts_sample (
    id INT
)
PARTITIONED BY (year INT, month STRING);


Writing task1_create_managed_table.hql


In [6]:
!hive -f task1_create_managed_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
Time taken: 12.926 seconds
OK
Time taken: 0.697 seconds


In [7]:
%%writefile task1_insert_table.hql
USE stackoverflow_;

SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.error.on.empty.partition=true;

-- Filling managed posts table from external one

FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT id, year, month;


Writing task1_insert_table.hql


In [8]:
!hive -f task1_insert_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
Time taken: 10.001 seconds
Query ID = jovyan_20210125065659_7a358e60-c783-4ac8-a7a6-4d280998e1d4
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1611556010983_0001, Tracking URL = http://172.17.0.2:8088/proxy/application_1611556010983_0001/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1611556010983_0001
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2021-01-25 06:57:14,646 Stage-1 map = 0%,  reduce = 0%
2021-01-25 06:57:45,911 Stage-1 map = 1%,  reduce = 0%, Cumulative CPU 26.61 sec
2021-01-25 06:58:16,359 Stage-1 map = 2%,  reduce = 0%, Cumulative CPU 57.52 sec
2021-01-25 06:58:28,100 Stage-1 map = 3%,  reduce = 0%, Cumulative CPU 69.39 sec
2021-01-25 06:58:45,729 Stage-1 map = 4%,  reduce = 0%, Cumulative CPU 87

2021-01-25 07:42:37,624 Stage-1 map = 88%,  reduce = 0%, Cumulative CPU 2656.8 sec
2021-01-25 07:43:25,184 Stage-1 map = 89%,  reduce = 0%, Cumulative CPU 2704.4 sec
2021-01-25 07:44:01,631 Stage-1 map = 90%,  reduce = 0%, Cumulative CPU 2740.42 sec
2021-01-25 07:44:55,286 Stage-1 map = 91%,  reduce = 0%, Cumulative CPU 2794.47 sec
2021-01-25 07:45:18,909 Stage-1 map = 92%,  reduce = 0%, Cumulative CPU 2818.33 sec
2021-01-25 07:45:49,773 Stage-1 map = 93%,  reduce = 0%, Cumulative CPU 2848.31 sec
2021-01-25 07:46:49,391 Stage-1 map = 94%,  reduce = 0%, Cumulative CPU 2908.56 sec
2021-01-25 07:47:31,853 Stage-1 map = 95%,  reduce = 0%, Cumulative CPU 2950.49 sec
2021-01-25 07:48:06,925 Stage-1 map = 96%,  reduce = 0%, Cumulative CPU 2986.31 sec
2021-01-25 07:49:01,576 Stage-1 map = 97%,  reduce = 0%, Cumulative CPU 3040.13 sec
2021-01-25 07:49:31,575 Stage-1 map = 98%,  reduce = 0%, Cumulative CPU 3070.07 sec
2021-01-25 07:50:07,612 Stage-1 map = 99%,  reduce = 0%, Cumulative CPU 3106.1

In [13]:
%%writefile task1_watch_new_table.hql
USE stackoverflow_;

SELECT * FROM posts_sample LIMIT 10;

Overwriting task1_watch_new_table.hql


In [14]:
!hive -f task1_watch_new_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
Time taken: 16.993 seconds
OK
1394	2008	2008-08
3543	2008	2008-08
4521	2008	2008-08
8689	2008	2008-08
9062	2008	2008-08
14671	2008	2008-08
16307	2008	2008-08
18780	2008	2008-08
18929	2008	2008-08
19668	2008	2008-08
Time taken: 4.361 seconds, Fetched: 10 row(s)


In [19]:
%%writefile task1_result.hql
USE stackoverflow_;

-- Count posts by months
SELECT year, month, COUNT(*) as count 
FROM posts_sample 
GROUP BY year, month ORDER BY month
LIMIT 2,1;

Overwriting task1_result.hql


In [20]:
!hive -f task1_result.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
OK
Time taken: 14.285 seconds
Query ID = jovyan_20210125075453_062c34fd-7103-4b3a-81a5-2de1656056e6
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1611556010983_0002, Tracking URL = http://172.17.0.2:8088/proxy/application_1611556010983_0002/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1611556010983_0002
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2021-01-25 07:55:11,706 Stage-1 map = 0%,  reduce = 0%
2021-01-25 07:55:19,250 St

## Step 2. Submission part. Do not touch!! And simple run all cells below!

In [ ]:
!cat task1_create_external_table.hql > task1.hql
!cat task1_create_managed_table.hql >> task1.hql
!cat task1_insert_table.hql >> task1.hql
!cat task1_result.hql >> task1.hql

In [ ]:
!cat task1.hql

In [ ]:
%%javascript

$(document).ready(function() {
    console.log('Ready');
    
    
    function is_hive_command(list_tokens) {
        return list_tokens.indexOf('hive') > -1 && 
             list_tokens.indexOf('f') > -1 &&
             list_tokens.indexOf('-') > -1 && 
             list_tokens.indexOf('!') > -1 &&
             list_tokens.indexOf('hql') > -1 && 
             list_tokens.indexOf('writefile') == -1;
    } 
    
    function collectText(input_tag) {

        var result_string = [];
        $.each($(input_tag).children(), function(index, child) {
            result_string.push($(child).text());
        });
        return [result_string, is_hive_command(result_string)];
    };
    
    var filtered_results = $(".cell.code_cell.rendered").filter(function(index, element) {
        var out = collectText($(element).find('.CodeMirror-line').find('span'));
        console.log(out);
        return collectText($(element).find('.CodeMirror-line').find('span'))[1];
    });
    $(filtered_results).remove();
});

In [ ]:
%%bash
hive -f task1.hql